Building the scoring matrix with given sequences and scoring scheme.

In [1]:
import numpy as np
import pandas as pd

# Sequences to be aligned
x = "GGCAGTA"
y = "GATGCGCAG"

# Scoring Criteria
match = 2
mismatch = -3
gap = -1


# From here the function definatioin will start.
nx = len(x)      # no. of columns, with x as one of the sequence
ny = len(y)      # no. of rows, with y as the other sequence

# Initialization of the 2D matrix.
F = np.zeros((nx + 1, ny + 1))

# Gap ko ek positive no. assume kiya jaata hai, and uska net effect is -ve (by placing - before it)
F[:, 0] = np.linspace(0, gap * nx, nx + 1)
F[0, :] = np.linspace(0, gap * ny, ny + 1)


# Till here, matrix with its 1st col and row is obtained.

# Pointers to trace through an optimal alignment.
# Matrix filling.
for i in range(1, nx + 1):
    for j in range(1, ny + 1):
        #complete the code
        # Calculating loss due to mismatch :
        resultant_if_mismatch = F [i-1, j-1] + mismatch
        resultant_if_gap = max([F [i, j-1] + gap, F [i-1, j] + gap])

        if (y[j-1] == x[i-1]):
            # Checking the case of Match
            F[i, j] = F [i-1, j-1] + match

        else :
            # Comparing whether the loss due to mismatch is more or due to upper/lower gaps.
            # Cell should always contain the maximum possible value.
            F[i, j] = max ([resultant_if_gap, resultant_if_mismatch])


Score = F[nx, ny] 

# Method 1 : Printing row by row with better formatting
# for row in F:
#     print(' '.join(map(str, row)))

# Method 2 : Or using NumPy's formatting options
# np.set_printoptions(precision=2, suppress=True)
# print(F)

# Method 3 : Print scoring matrix using pandas DataFrame
print("Scoring Matrix:")
df = pd.DataFrame(F, index=['-'] + list(x), columns=['-'] + list(y))
print(df)
print("\nThe Score for all the Optimal Alignments is", Score)

Scoring Matrix:
     -    G    A    T    G    C    G    C    A    G
-  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0 -7.0 -8.0 -9.0
G -1.0  2.0  1.0  0.0 -1.0 -2.0 -3.0 -4.0 -5.0 -6.0
G -2.0  1.0  0.0 -1.0  2.0  1.0  0.0 -1.0 -2.0 -3.0
C -3.0  0.0 -1.0 -2.0  1.0  4.0  3.0  2.0  1.0  0.0
A -4.0 -1.0  2.0  1.0  0.0  3.0  2.0  1.0  4.0  3.0
G -5.0 -2.0  1.0  0.0  3.0  2.0  5.0  4.0  3.0  6.0
T -6.0 -3.0  0.0  3.0  2.0  1.0  4.0  3.0  2.0  5.0
A -7.0 -4.0 -1.0  2.0  1.0  0.0  3.0  2.0  5.0  4.0

The Score for all the Optimal Alignments is 4.0


Retracing to find one of the optimal alignments.

In [2]:
# Assumptions for the below code.
# my code by deault moves the upper cell rather than the cell to the left, in case of 2 possible directions from that point; so in the other case, 
# we will by default move to the lefftward cell when we have to choose between 2 paths.


P = np.zeros((nx + 1, ny + 1), dtype=int)
# P[:, 0] = 3
# P[0, :] = 4


# Retracing
Paths = []
i = nx
j = ny

# List of cell indices which are to be re-visited.
cell_revisit = []

# List of cells, which were at the verge of opening 2 paths.
cells_opening_2_paths = []


rx = []
ry = []
path_traversed = ""
# print (i, j)
while i>0 or j>0:   
    # print("Started", end = "...")
    if (x[i-1] == y[j-1]):
        # Case when its a match
        rx.append(x[i-1]) 
        ry.append(y[j-1])
      
        # Adding cell to the path traversed.
        path_traversed += "({}, {})".format(i, j)

        # Shifted the pointer to previous diagonal cell.
        i = i-1
        j = j-1
    # we know its a mismatch/gap, hence checking which is the most suitable nearby cell, from where the transition would have taken.
    elif (F[i-1, j-1] + mismatch == F[i, j]):
        # Case when its a mismatch
        rx.append(x[i-1])
        ry.append(y[j-1])
      
        # Adding cell to the path traversed.
        path_traversed += "({}, {})".format(i, j)
        
        # Checking if the adjascent cells are also having the same score.
        if (F[i, j-1] + gap == F[i, j]):
          cell_revisit.append([i, j-1])
          cells_opening_2_paths.append([i, j])
          
        if (F[i-1, j] + gap == F[i, j]):
          cell_revisit.append([i-1, j])
          cells_opening_2_paths.append([i, j])
        
        # Shifted the pointer to previous diagonal cell.
        i = i-1
        j = j-1
    elif (F[i, j-1] + gap == F[i, j]):
        # Case of gap when transition made upwards
        rx.append("-")
        ry.append(y[j-1])
              
        # Adding cell to the path traversed.
        path_traversed += "({}, {})".format(i, j)

        # Checking if the adjascent cells are also having the same score.
        if (F[i-1, j] + gap == F[i, j]):
          cell_revisit.append([i-1, j])  
          cells_opening_2_paths.append([i, j])
        
        # Shifting the pointer
        i = i
        j -= 1
    elif (F[i-1, j] + gap == F[i, j]):
        # Case of gap when transition made leftwards
        rx.append(x[i-1])
        ry.append("-")
      
        # Adding cell to the path traversed.
        path_traversed += "({}, {})".format(i, j)

        # Checking if the adjascent cells are also having the same score.
        if (F[i, j-1] + gap == F[i, j]):
          cell_revisit.append([i, j-1])
          cells_opening_2_paths.append([i, j])
        
        # Shifing the pointer.
        i -= 1
        j = j
    # print("Done", "({}, {})".format(i, j))

print()
# Reverse the strings.
rx = ''.join(rx)[::-1]
ry = ''.join(ry)[::-1]
l = ""
for n in range (len(rx)):
  if (rx[n] ==  ry[n]):
    l+="|"
  else :
    l+=" "
print ("First of the many possible optimal alignments :" + "\n" + rx + "\n" + l + "\n" + ry)

# print ("\nThe cell positions traversed by the pointer for tracing the first optimal alignment is :")
# print (path_traversed)

# print("The cells which are to be revisited are : ", cell_revisit)
# print("The cells from where the two path divergence starts : ", cells_opening_2_paths)



First of the many possible optimal alignments :
G--GCAGT-A-
|  || |  | 
GATGC-G-CAG


Using Dynamic Programming to find all the possible alignments.

In [3]:
# Since the above code is not efficient for writing all the possible optimal alignments, hence writing a recursive approach.

def retracing(F, x, y, i, j, alignment, possible_alignments, match, mismatch, gap):
    if i == 0 and j == 0:
        if (alignment[0][::-1], alignment[1][::-1], F[len(x), len(y)]) not in possible_alignments:
            possible_alignments.append((alignment[0][::-1], alignment[1][::-1], F[len(x), len(y)]))
        return

    if i > 0 and F[i, j] == F[i-1, j] + gap:
        retracing(F, x, y, i-1, j, (alignment[0] + x[i-1], alignment[1] + '-'), possible_alignments, match, mismatch, gap)

    if j > 0 and F[i, j] == F[i, j-1] + gap:
        retracing(F, x, y, i, j-1, (alignment[0] + '-', alignment[1] + y[j-1]), possible_alignments, match, mismatch, gap)

    if i > 0 and j > 0 and F[i, j] == F[i-1, j-1] + (match if x[i - 1] == y[j - 1] else mismatch):
        retracing(F, x, y, i-1, j-1, (alignment[0] + x[i-1], alignment[1] + y[j-1]), possible_alignments, match, mismatch, gap)


possible_alignments = []
retracing(F, x, y, nx, ny, ('', ''), possible_alignments, match, mismatch, gap)

print()
print("Now, Printing All The  Possible Optimal Alignments:\n")
for idx in range(len(possible_alignments)):
    alignment = possible_alignments[idx]
    # A list which will show connections betwenn the two sequences.
    mid = ""
    for k in range(len(alignment[0])):
      if alignment[0][k] == alignment[1][k] : 
        mid += "|"
      else :
        mid += " "
    print (f"Alignment {idx + 1}:\n {alignment[0]} \n {mid} \n {alignment[1]} | Score: {alignment[2]}\n")



Now, Printing All The  Possible Optimal Alignments:

Alignment 1:
 G--GC--AGTA 
 |  ||  ||   
 GATGCGCAG-- | Score: 4.0

Alignment 2:
 G--G--CAGTA 
 |  |  |||   
 GATGCGCAG-- | Score: 4.0

Alignment 3:
 G----GCAGTA 
 |    ||||   
 GATGCGCAG-- | Score: 4.0

Alignment 4:
 ---G-GCAGTA 
    | ||||   
 GATGCGCAG-- | Score: 4.0

Alignment 5:
 G--GCAG-TA- 
 |  || |  |  
 GATGC-GC-AG | Score: 4.0

Alignment 6:
 G--GCAGT-A- 
 |  || |  |  
 GATGC-G-CAG | Score: 4.0

